In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as pyo
import plotly.graph_objs as go

In [2]:
df = pd.read_excel(r'C:\Users\Mi Notebook\Downloads\US State Wind Energy Facts 2018.xlsx')

In [3]:
df.drop(df.index[40:], inplace=True)

In [4]:
df['$ per Turbine'] = (df['Total Investment ($ Millions)']/df['# of Wind Turbines']).round(2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 8 columns):
Ranking                          40 non-null float64
State                            40 non-null object
Installed Capacity (MW)          40 non-null int64
Equivalent Homes Powered         40 non-null float64
Total Investment ($ Millions)    40 non-null float64
Wind Projects Online             40 non-null int64
# of Wind Turbines               40 non-null float64
$ per Turbine                    40 non-null float64
dtypes: float64(5), int64(2), object(1)
memory usage: 2.8+ KB


In [6]:
abr = pd.read_excel(r'C:\Users\Mi Notebook\Desktop\US_STATES_Abbreviations.xlsx')

In [67]:
# df['State'] = df['State'].str.encode('utf-8')
# abr['State'] = abr['State'].str.encode('utf-8')
df['State'] = df['State'].str.strip()
abr['State'] = abr['State'].str.strip()
result = pd.merge(df, abr, how='inner', on='State')

In [68]:
result['Abbreviation'] = result['Abbreviation'].str.strip()
result.head()

,Ranking,State,Installed Capacity (MW),Equivalent Homes Powered,Total Investment ($ Millions),Wind Projects Online,# of Wind Turbines,$ per Turbine,Abbreviation
0,1.0,TEXAS,23262,6235000.0,42000.0,136,12750.0,3.29,TX
1,2.0,OKLAHOMA,7495,2268000.0,13700.0,45,3717.0,3.69,OK
2,3.0,IOWA,7312,1935000.0,14200.0,107,4145.0,3.43,IA
3,4.0,CALIFORNIA,5686,1298000.0,12600.0,104,6972.0,1.81,CA
4,5.0,KANSAS,5110,1719000.0,9400.0,35,2795.0,3.36,KS


In [69]:
geo_sites = pd.read_excel(r'C:\Users\Mi Notebook\Desktop\Capitals_Geo_Pos.xlsx')
geo_sites['STATE_ABBR'] = geo_sites['STATE_ABBR'].str.strip()

In [70]:
result['Abbreviation'] = result['Abbreviation'].str.encode('utf-8')
geo_sites['STATE_ABBR'] = geo_sites['STATE_ABBR'].str.encode('utf-8')

In [71]:
geo_sites.head()

,STATE,STATE_ABBR,LATITUDE,LONGITUDE
0,ALABAMA,b'AL',32.806671,-86.791130
1,ALASKA,b'AK',61.370716,-152.404419
2,ARIZONA,b'AZ',33.729759,-111.431221
3,ARKANSAS,b'AR',34.969704,-92.373123
4,CALIFORNIA,b'CA',36.116203,-119.681564


In [72]:
result = pd.merge(result, geo_sites, how='inner', left_on='Abbreviation', right_on='STATE_ABBR')

In [88]:
columns_to_int = ['Equivalent Homes Powered', 'Total Investment ($ Millions)','# of Wind Turbines']

In [89]:
for col in columns_to_int:
    result[col] = result[col].astype(int)

In [92]:
result.head()

,Ranking,State,Installed Capacity (MW),Equivalent Homes Powered,Total Investment ($ Millions),Wind Projects Online,# of Wind Turbines,$ per Turbine,Abbreviation,LATITUDE,LONGITUDE,text
0,1.0,TEXAS,23262,6235000,42000,136,12750,3.29,TX,31.054487,-97.563461,Total Investment ($ mln):42000<br>Number of Wi...
1,2.0,OKLAHOMA,7495,2268000,13700,45,3717,3.69,OK,35.565342,-96.928917,Total Investment ($ mln):13700<br>Number of Wi...
2,3.0,IOWA,7312,1935000,14200,107,4145,3.43,IA,42.011539,-93.210526,Total Investment ($ mln):14200<br>Number of Wi...
3,4.0,CALIFORNIA,5686,1298000,12600,104,6972,1.81,CA,36.116203,-119.681564,Total Investment ($ mln):12600<br>Number of Wi...
4,5.0,KANSAS,5110,1719000,9400,35,2795,3.36,KS,38.526600,-96.726486,Total Investment ($ mln):9400<br>Number of Win...


In [74]:
result.drop(columns=['STATE','STATE_ABBR'], inplace=True)

In [76]:
result['Abbreviation'] = result['Abbreviation'].str.decode('utf-8') 

In [91]:
result['text'] = 'Total Investment ($ mln):' + result['Total Investment ($ Millions)'].astype(str)  + '<br>' + 'Number of Wind Turbines:'+ result['# of Wind Turbines'].astype(str) + '<br>' + 'Equivalent Homes Powered:'+ result['Equivalent Homes Powered'].astype(str) + '<br>' + result['State']   

In [94]:
trace1 = [ dict(
        type='choropleth',
        locations = result['Abbreviation'],
        z = result["Installed Capacity (MW)"].astype(float),
        locationmode = 'USA-states',
        text = '',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Installed Capacity (MW)")
        ) ]

trace2 = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = result["LONGITUDE"],
        lat = result["LATITUDE"],
        hoverinfo = 'text',
        text = result['text'],
        mode = 'markers',
        marker = dict(
            size=result['$ per Turbine']*3,
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='blue'
            )
        ))]
    
layout = dict(
        title = 'US State Wind Energy Facts 2018',
        showlegend = False,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )    
    
    
fig = dict( data=trace1 + trace2, layout=layout )

pyo.plot(fig, filename='US_CHOROPLETH_PLOTLY_NEW.html')

'file://C:\\Users\\Mi Notebook\\US_CHOROPLETH_PLOTLY_NEW.html'